In [ ]:
# default_exp cleaner

# Cleaner

> The clearners are the last but not least blocks of web2dataset. Their goal is to purge and clean the dataset.

Example of cleaner (not yet implemented):

* delete double (based on hash)
* delete image with low resolution
* ethic base purger, how to ?

There are two kind of cleaner, the one that work on metadata that are called before downloading the image and the one that work on image and are called after


In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
# hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# export
from abc import ABC, abstractmethod
from functools import wraps
from typing import List

from web2dataset.document import Document
from web2dataset.utils_data import get_metadata_path, load_docs,get_images_path

In [ ]:
import os
import shutil

# MetaDataCleaner

In [ ]:
# export
class MetaDataCleanerError(ValueError):
    pass

a cleaner should delete docs not create them, so we verify than we did not create new docs with this wrapper

In [ ]:
# export
def check_no_docs_creation(f):
    @wraps(f)
    def wrapper(self, docs: List[Document]) -> List[Document]:
        new_docs = f(self, docs)
        if len(new_docs) > len(docs):
            raise MetaDataCleanerError(
                f"the cleaner should not create more docs than originaly. There were before {len(docs)} docs and there are now {len(new_docs)} docs"
            )
        return new_docs

    return wrapper

Here is the abstract class for the meta data cleaner. It only operate on documents not images

In [ ]:
# export
class MetaDataCleaner(ABC):
    @abstractmethod
    @check_no_docs_creation
    def clean(self, docs: List[Document]) -> List[Document]:
        pass

here is a basic cleaner that is mainly used for testing

In [ ]:
# export
class IdentityCleaner:
    @check_no_docs_creation
    def clean(self, docs: List[Document]) -> List[Document]:
        return docs

In [ ]:
docs = [
    Document(origin="", image_url="https://image/bike"),
    Document(origin="", image_url="https://image/bike"),
    Document(origin="", image_url="https://image/bmx"),
]

In [ ]:
cleaner = IdentityCleaner()
docs = cleaner.clean(docs)

## Duplicate cleaner

This MetaDataDuplicateCleaner delete any duplicate, i.e document with the same src image to avoid downloading twice the same image.
It is different from the ImageDuplicateCleaner, will delete two identical image after the donwload, this images could come from two different sources be still be the same

In [ ]:
# export
class DuplicateCleaner(MetaDataCleaner):
    @check_no_docs_creation
    def clean(self, docs: List[Document]) -> List[Document]:
        url_doc = {
            doc.image_url: doc for doc in docs
        }  # first we create a dict with image url as key because we want to keep only one doc per image_rul
        return list(url_doc.values())

In [ ]:
docs = [
    Document(origin="", image_url="https://image/bike"),
    Document(origin="", image_url="https://image/bike"),
    Document(origin="", image_url="https://image/bmx"),
]

In [ ]:
url = [doc.image_url for doc in docs]
len(url), len(set(url))

(3, 2)

as we can see in this list of doc there are 3 url but only two of them are different. Let's fix it

In [ ]:
cleaner = DuplicateCleaner()

In [ ]:
docs2 = cleaner.clean(docs)
url = [doc.image_url for doc in docs2]

assert len(url) == len(set(url))

# ImageCleaner

Image cleaner operatire directly on image and are called after the downloading step wheras the MetaDataCleaner are called after the search

In [ ]:
# export
class ImageCleaner(ABC):
    """
    Abstract class to delete
    """

    def __init__(self, path: str):
        self.path = path

    @abstractmethod
    def clean(self):
        pass

## Duplicate image cleaner

In [ ]:
class DuplicateImageCleaner(ImageCleaner):
    """
    Delete duplicate image by computing the hash of each image and delete doublon
    args:
        path: str. A string containing the path to the folder (where we saved a search and download images)
    """

    def __init_(self, path: str):
        super().__init__(path)
        self.documents = load_docs(self.path)
        self.image_path = get_images_path(self.path)

    def _extract_hash(self):
        pass
        s
    def clean(self):
        pass

let's try it out

In [ ]:
from web2dataset.downloader import BasicDownloader
from web2dataset.searcher import BasicSearcher

In [ ]:
path_test = "/tmp/my_search_test_cleaner"
if os.path.isdir(path_test):
    shutil.rmtree(path_test)

In [ ]:
searcher = BasicSearcher("", 2)
searcher.search()
searcher.save(path_test)

In [ ]:
downloader = BasicDownloader(path_test)
downloader.download()

In [ ]:
cleaner = DuplicateImageCleaner(path_test)